In [1]:
import pandas as pd
import urllib
import os
import httplib2

from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage

In [2]:
def download(url,filename):
    #filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.urlretrieve(url, filename)

In [3]:
url =  'http://image-net.org/archive/words.txt'
filename = 'all_imagenet_synsets_and_descriptions.txt'

download(url,filename) 

df = pd.read_csv(filename, sep=r'\t',engine='python',
                          names=['wnid','description'],na_filter=False)
print 'Original wnids and dscriptions form Image-Net'
df.head()

Original wnids and dscriptions form Image-Net


,wnid,description
0,n00001740,entity
1,n00001930,physical entity
2,n00002137,"abstraction, abstract entity"
3,n00002452,thing
4,n00002684,"object, physical object"


In [4]:
filename = 'all_wnids.csv'
sep= ';'
if not os.path.exists(filename):
    df_temp = pd.concat([pd.Series(row['wnid'], row['description'].split(','))              
                        for _, row in df.iterrows()]).reset_index()
    all_wnids = df_temp.rename(columns={'index':'description',0:'wnid'})
    all_wnids['description'] = all_wnids['description'].str.lower().str.strip()
    all_wnids.to_csv('all_wnids.csv',sep=sep,index=False)
else: 
    all_wnids = pd.read_csv(filename,sep=sep)
print 'All wnids descriptions split in one column'
all_wnids.head()

All wnids descriptions split in one column


,description,wnid
0,entity,n00001740
1,physical entity,n00001930
2,abstraction,n00002137
3,abstract entity,n00002137
4,thing,n00002452


In [5]:
len(all_wnids)

146347

In [6]:
print all_wnids.ix[all_wnids.description == 'fruit'], '\n'

print all_wnids.ix[all_wnids.description =='vegetable'], '\n'

veg_wnid = 'n07707451'
fruit_wnid = 'n13134947'
# veg_wnid = all_wnids.ix[all_wnids.description =='vegetable'].values[0,1]
# fruit_wnid = all_wnids.ix[all_wnids.description =='fruit'].values[1,1]

print 'Vegetables Master Synset:','\t',veg_wnid
print 'Fruits Master Synset:','\t\t',fruit_wnid

       description       wnid
44632        fruit  n04612722
68317        fruit  n07294550
126326       fruit  n13134947 

       description       wnid
72367    vegetable  n07707451
115918   vegetable  n12212361 

Vegetables Master Synset: 	n07707451
Fruits Master Synset: 		n13134947


In [7]:
url1 = 'http://www.image-net.org/api/text/wordnet.structure.hyponym?wnid='
param1 = '&full=1'

response = urllib.urlopen(url1+veg_wnid+param1)
veg_wnids = pd.DataFrame(response.read().replace('-','').splitlines())
veg_wnids.columns = ['wnid']
veg_wnids['group']='vegetables'
print 'All subcategories from', veg_wnid
print veg_wnids.head()

response = urllib.urlopen(url1+fruit_wnid+param1)
fruit_wnids = pd.DataFrame(response.read().replace('-','').splitlines())
fruit_wnids.columns = ['wnid']
fruit_wnids['group']='fruits'
print 'All subcategories from', fruit_wnid
print fruit_wnids.head()

All subcategories from n07707451
        wnid       group
0  n07707451  vegetables
1  n07708124  vegetables
2  n07708398  vegetables
3  n07709046  vegetables
4  n07725158  vegetables
All subcategories from n13134947
        wnid   group
0  n13134947  fruits
1  n07745940  fruits
2  n07738105  fruits
3  n07738224  fruits
4  n07739035  fruits


In [8]:
all_fr_veg_wnids = veg_wnids.append(fruit_wnids)
all_fr_veg_wnids.shape

(497, 2)

In [9]:
# If modifying these scopes, delete your previously saved credentials
# at ~/.credentials/sheets.googleapis.com-python-quickstart.json
SCOPES = 'https://www.googleapis.com/auth/spreadsheets.readonly'
CLIENT_SECRET_FILE = '../client_secret.json'
APPLICATION_NAME = 'Test'


def get_credentials():
    """Gets valid user credentials from storage.

    If nothing has been stored, or if the stored credentials are invalid,
    the OAuth2 flow is completed to obtain the new credentials.

    Returns:
        Credentials, the obtained credential.
    """
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        os.makedirs(credential_dir)
    credential_path = os.path.join(credential_dir,
                                   'sheets.googleapis.com-python-quickstart.json')

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = APPLICATION_NAME
        if flags:
            credentials = tools.run_flow(flow, store, flags)
        else: # Needed only for compatibility with Python 2.6
            credentials = tools.run(flow, store)
        print 'Storing credentials to ' + credential_path 
    return credentials


"""Shows basic usage of the Sheets API.

ingrdients spreadsheet:
https://docs.google.com/spreadsheets/d/1LHojuRe1U7nhJUG-m7aMfA4BNQX-BmbmGlCSErE1NIg/edit
"""
credentials = get_credentials()
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                'version=v4')
service = discovery.build('sheets', 'v4', http=http,
                          discoveryServiceUrl=discoveryUrl)

spreadsheetId = '1LHojuRe1U7nhJUG-m7aMfA4BNQX-BmbmGlCSErE1NIg'
rangeName = 'All_in_one'
result = service.spreadsheets().values().get(
    spreadsheetId=spreadsheetId, range=rangeName).execute()
values = result.get('values', [])

if not values:
    print 'No data found.'

In [10]:
scoodit_ingredients = pd.DataFrame(values[1:])
#                                    ,columns=values[0])\
#                         .dropna(subset=['Categorie1'])\
scoodit_ingredients.head()

,0,1,2,3,4,5
0,Fruit,Apple,None,None,None,None
1,Fruit,Apricot,None,None,None,None
2,Fruit,Banana,None,None,None,None
3,Fruit,Blackberry,None,None,None,None
4,Fruit,Blueberry,None,None,None,None


In [ ]:
for ind, ing in scoodit_ingredients.iterrows():
    if  

In [ ]:
for col in scoodit_ingredients.columns:
    scoodit_ingredients[col]= scoodit_ingredients[col].str.lower().str.strip()
scoodit_ingredients.dropna(subset=['Categorie1'],inplace=True)

In [ ]:
scoodit_ingredients.shape

In [ ]:
scoodit_ingredients.head()

In [ ]:
df1 = pd.merge(all_fr_veg_wnids,all_wnids,how='inner',on='wnid')
print df1.shape
df1.head()

In [ ]:
print all_fr_veg_wnids.shape
all_fr_veg_wnids.head()

In [ ]:
print all_wnids.shape
all_wnids.head()

In [ ]:
merged_set = set()
orig_set = set()
merged_df = scoodit_ingredients
for col in scoodit_ingredients.columns[1:2]:
    print col
    merged_df = pd.merge(merged_df,all_wnids,left_on=col,right_on='description')\
    .sort_values('wnid').drop_duplicates(subset='wnid')
    
    merged_set = merged_set.union(set(merged_df[col]))
    orig_set = set(scoodit_ingredients[col])
    missing = [i for i in orig_set if i not in merged_set]
    missing_ingredients = scoodit_ingredients.ix[scoodit_ingredients[col].isin(missing)]

In [ ]:
len(missing_ingredients)

In [ ]:
len(missing)

In [ ]:
merged_df

In [ ]:
print test.shape
test.head()

In [ ]:
merged_set = set(test.Categorie1)
orig_set = set(scoodit_ingredients.Categorie1)

[i for i in orig_set if i not in merged_set]

In [ ]:
all_wnids.ix[all_wnids['description'].str.contains('white radish') == True]

In [ ]:
test2 = pd.melt(scoodit_ingredients, id_vars=["Group"], 
                  var_name="Date", value_name="Value").dropna(subset=['Value']).drop(['Date'],axis=1)

In [ ]:
test2.ix[test2['Group'] == 'seafood']